In [1]:
#Importing Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Loading the dataset into a data frame
df_fd = pd.read_csv('../data/fashion dataset.csv')
df_bd = pd.read_csv('../data/fashion brand details.csv')

In [3]:
df_fd.head(10)

,p_id,name,price,colour,brand,ratingCount,avg_rating,description,p_attributes
0,1518329.0,Dupatta Bazaar White Embroidered Chiffon Dupatta,899.0,White,Dupatta Bazaar,1321.0,4.548827,White embroidered&nbsp;dupattaChiffon<br>Hand-...,"{'Occasion': 'Daily', 'Pattern': 'Embroidered'..."
1,5829334.0,Roadster Women Mustard Yellow Solid Hooded Swe...,1199.0,Mustard,Roadster,5462.0,4.313255,"Mustard yellow solid sweatshirt, has a hood, t...","{'Body Shape ID': '443,424,324', 'Body or Garm..."
2,10340119.0,Inddus Peach-Coloured & Beige Unstitched Dress...,5799.0,Peach,Inddus,145.0,4.068966,Peach-Coloured and beige woven design unstitch...,"{'Bottom Fabric': 'Cotton Blend', 'Bottom Patt..."
3,10856380.0,SASSAFRAS Women Black Parallel Trousers,1499.0,Black,SASSAFRAS,9124.0,4.147523,"Black solid woven high-rise parallel trousers,...","{'Add-Ons': 'NA', 'Body Shape ID': '424', 'Bod..."
4,12384822.0,Kotty Women Black Wide Leg High-Rise Clean Loo...,1999.0,Black,Kotty,12260.0,4.078467,"Black dark wash 4-pocket high-rise jeans, clea...","{'Add-Ons': 'NA', 'Brand Fit Name': 'NA', 'Clo..."
5,12742100.0,KASSUALLY Women Black & Pink Printed Basic Jum...,2199.0,Black,KASSUALLY,6297.0,4.349214,"Black printed basic jumpsuit, has a V-neck, lo...",{'Body or Garment Size': 'Garment Measurements...
6,13842966.0,Sassafras Brown & Red Geometric Printed George...,1499.0,Brown,SASSAFRAS,7358.0,4.395352,<ul><li>Brown and red crop wrap top</li><li>Pr...,"{'Body Shape ID': '333,424', 'Body or Garment ..."
7,14021452.0,Sera Women Multicoloured Printed Tie-Up Shrug,1494.0,Multi,Sera,750.0,4.288000,"Brown and blue printed tie-up longline shrug, ...",{'Body or Garment Size': 'To-Fit Denotes Body ...
8,14063026.0,Tokyo Talkies Women Black Solid Regular Shorts,699.0,Black,Tokyo Talkies,1856.0,4.530711,"Black solid mid-rise regular shorts, has 2 poc...",{'Body or Garment Size': 'To-Fit Denotes Body ...
9,14324806.0,Anouk Stylish Black Solid Ready to Wear Leheng...,4699.0,Black,Anouk,84.0,3.809524,Stay fashionable in any season when you don th...,"{'Blouse Closure': 'Zip', 'Blouse Fabric': 'Vi..."


In [4]:
df_bd.head(10)

,brand_id,brand_name
0,1,513
1,2,109F
2,3,20Dresses
3,4,250 Designs
4,5,3Pin
5,6,6Y Collective
6,7,98 Degree North
7,8,9Teenagain
8,9,A Little Fable
9,10,A.T.U.N.


In [5]:
df_fd.tail(10)

,p_id,name,price,colour,brand,ratingCount,avg_rating,description,p_attributes
14319,13646266.0,High Star Women Mustard Yellow Solid Stretchab...,1999.0,Mustard,High Star,NaN,NaN,"Mustard Yellow solid jacket, has a spread coll...","{'Add-Ons': 'NA', 'Body Shape ID': '443,333,42..."
14320,14235030.0,Indo Era Women Black Floral Printed Pure Cotto...,3499.0,Black,Indo Era,12.0,4.583333,Black and green printed Kurta with Trousers wi...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
14321,16379656.0,Vero Moda Women Grey & Green Colourblocked Swe...,2299.0,Grey,Vero Moda,NaN,NaN,Grey and green colourblocked sweatshirt has a ...,"{'Body Shape ID': '443,424,324', 'Body or Garm..."
14322,16835446.0,Allen Solly Woman Women Blue Skinny Fit Jeans,1999.0,Blue,Allen Solly Woman,49.0,4.346939,"<ul> <li> Light shade, no fade blue jeans </l...","{'Add-Ons': 'NA', 'Body or Garment Size': 'To-..."
14323,16944668.0,Berrylush Women Pretty Pink Romantic Florals Top,1799.0,Pink,Berrylush,7.0,3.000000,Look cool and stylish when you slip into this ...,"{'Body Shape ID': '333,424', 'Body or Garment ..."
14324,17029604.0,The Chennai Silks Pink & Silver-Toned Floral Z...,3999.0,Pink,The Chennai Silks,NaN,NaN,<b>Design Details </b><ul><li>Pink and silver-...,"{'Better Cotton Initiative': 'Regular', 'Blous..."
14325,17600212.0,Kinder Kids Girls Blue & Green Printed Foil Pr...,2050.0,Blue,Kinder Kids,NaN,NaN,"Blue and green printed lehenga choli, has foi...","{'Blouse Closure': 'NA', 'Blouse Fabric': 'Cot..."
14326,18159266.0,KLOTTHE Women Green & Black Floral Printed Pal...,1659.0,Green,KLOTTHE,NaN,NaN,<ul> <li> Green and black woven palazzos </li>...,{'Body or Garment Size': 'To-Fit Denotes Body ...
14327,18921114.0,InWeave Women Red Printed A-Line Skirt,2399.0,Red,InWeave,NaN,NaN,"<p>Red printed A-line skirt, has drawstring cl...","{'Add-Ons': 'NA', 'Body Shape ID': '324,333,42..."
14328,19361058.0,BoStreet Women Navy Blue Tapered Fit Trousers,2599.0,Navy Blue,BoStreet,NaN,NaN,<ul> <li> Navy blue knitted trousers </li> <li...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,42..."


In [6]:
df_fd.tail(10)

,p_id,name,price,colour,brand,ratingCount,avg_rating,description,p_attributes
14319,13646266.0,High Star Women Mustard Yellow Solid Stretchab...,1999.0,Mustard,High Star,NaN,NaN,"Mustard Yellow solid jacket, has a spread coll...","{'Add-Ons': 'NA', 'Body Shape ID': '443,333,42..."
14320,14235030.0,Indo Era Women Black Floral Printed Pure Cotto...,3499.0,Black,Indo Era,12.0,4.583333,Black and green printed Kurta with Trousers wi...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
14321,16379656.0,Vero Moda Women Grey & Green Colourblocked Swe...,2299.0,Grey,Vero Moda,NaN,NaN,Grey and green colourblocked sweatshirt has a ...,"{'Body Shape ID': '443,424,324', 'Body or Garm..."
14322,16835446.0,Allen Solly Woman Women Blue Skinny Fit Jeans,1999.0,Blue,Allen Solly Woman,49.0,4.346939,"<ul> <li> Light shade, no fade blue jeans </l...","{'Add-Ons': 'NA', 'Body or Garment Size': 'To-..."
14323,16944668.0,Berrylush Women Pretty Pink Romantic Florals Top,1799.0,Pink,Berrylush,7.0,3.000000,Look cool and stylish when you slip into this ...,"{'Body Shape ID': '333,424', 'Body or Garment ..."
14324,17029604.0,The Chennai Silks Pink & Silver-Toned Floral Z...,3999.0,Pink,The Chennai Silks,NaN,NaN,<b>Design Details </b><ul><li>Pink and silver-...,"{'Better Cotton Initiative': 'Regular', 'Blous..."
14325,17600212.0,Kinder Kids Girls Blue & Green Printed Foil Pr...,2050.0,Blue,Kinder Kids,NaN,NaN,"Blue and green printed lehenga choli, has foi...","{'Blouse Closure': 'NA', 'Blouse Fabric': 'Cot..."
14326,18159266.0,KLOTTHE Women Green & Black Floral Printed Pal...,1659.0,Green,KLOTTHE,NaN,NaN,<ul> <li> Green and black woven palazzos </li>...,{'Body or Garment Size': 'To-Fit Denotes Body ...
14327,18921114.0,InWeave Women Red Printed A-Line Skirt,2399.0,Red,InWeave,NaN,NaN,"<p>Red printed A-line skirt, has drawstring cl...","{'Add-Ons': 'NA', 'Body Shape ID': '324,333,42..."
14328,19361058.0,BoStreet Women Navy Blue Tapered Fit Trousers,2599.0,Navy Blue,BoStreet,NaN,NaN,<ul> <li> Navy blue knitted trousers </li> <li...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,42..."


In [7]:
#Check for empty rows
df_fd.isna().all(axis=1).sum()

18

In [8]:
#Dropping all empty rows
df_fd.dropna(axis=0, how='all', inplace=True)
df_fd.isna().all(axis=1).sum()

0

In [9]:
df_bd.isna().all(axis=1).sum()

0

In [10]:
df_fd.reset_index(drop=True, inplace=True)

In [11]:
df_fd_desc = df_fd[['p_id','description']]
df_fd_p_attributes = df_fd[['p_attributes']]


In [12]:
print(df_fd_p_attributes.head(2))
print(df_fd_desc.head(2))

                                        p_attributes
0  {'Occasion': 'Daily', 'Pattern': 'Embroidered'...
1  {'Body Shape ID': '443,424,324', 'Body or Garm...
        p_id                                        description
0  1518329.0  White embroidered&nbsp;dupattaChiffon<br>Hand-...
1  5829334.0  Mustard yellow solid sweatshirt, has a hood, t...


In [14]:
df_fd.drop(['description'], axis = 1, inplace = True)
df_fd.drop(['p_attributes'], axis = 1, inplace = True)
df_fd.head(5)

,p_id,name,price,colour,brand,ratingCount,avg_rating
0,1518329.0,Dupatta Bazaar White Embroidered Chiffon Dupatta,899.0,White,Dupatta Bazaar,1321.0,4.548827
1,5829334.0,Roadster Women Mustard Yellow Solid Hooded Swe...,1199.0,Mustard,Roadster,5462.0,4.313255
2,10340119.0,Inddus Peach-Coloured & Beige Unstitched Dress...,5799.0,Peach,Inddus,145.0,4.068966
3,10856380.0,SASSAFRAS Women Black Parallel Trousers,1499.0,Black,SASSAFRAS,9124.0,4.147523
4,12384822.0,Kotty Women Black Wide Leg High-Rise Clean Loo...,1999.0,Black,Kotty,12260.0,4.078467


In [72]:
def split_feature(df):
    series = df.iloc[:,0]
    options = {}
    empty_id = []
    # Iterate over the column
    # for id, values  in series[series.notnull()].items():
    for id, values  in series.items():
    # Break each value into list of options
    # print(id, values)
        if type(values) == str:
            for key, value in eval(values).items():
                
                # Add the option as a column to result
                if not key in options.keys():
                    options[key] = [np.nan for x in range(df.shape[0])]#create a new column if it doesnt exist and set all value under it to empty
                # Mark the value in the option column as True
                options[key][id] = value
        else:
            empty_id.append(id)
        
    for x in range(len(empty_id)):
        for key in options.keys():
            options[key][empty_id[x]] = np.nan
                
    return pd.DataFrame(data = options)

In [67]:
# calling the spliting function
df_fd_p_split = split_feature(df_fd_p_attributes)
df_fd_p_split.head(2)

12146


,Occasion,Pattern,Print or Pattern Type,Body Shape ID,Body or Garment Size,Closure,Fabric,Hemline,Hood,Length,...,Fabric Purity,Fusion Wear,Pattern Size,Contact Brand or Retailer for pre-sales product queries,Colour Family,Collection Name,Better Cotton Initiative,About the Brand,How to care for your denim,What it does for your body
0,Daily,Embroidered,Floral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Casual,Solid,Solid,"443,424,324",To-Fit Denotes Body Measurements in,Zip,Cotton,Straight,Hooded,Regular,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
#merging the fashion dataset dataframe with the splitted dataframe
df_fd_full = pd.concat([df_fd, df_fd_p_split], axis=1, ignore_index = False)
df_fd_full.head(2)

,p_id,name,price,colour,brand,ratingCount,avg_rating,Occasion,Pattern,Print or Pattern Type,...,Fabric Purity,Fusion Wear,Pattern Size,Contact Brand or Retailer for pre-sales product queries,Colour Family,Collection Name,Better Cotton Initiative,About the Brand,How to care for your denim,What it does for your body
0,1518329.0,Dupatta Bazaar White Embroidered Chiffon Dupatta,899.0,White,Dupatta Bazaar,1321.0,4.548827,Daily,Embroidered,Floral,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5829334.0,Roadster Women Mustard Yellow Solid Hooded Swe...,1199.0,Mustard,Roadster,5462.0,4.313255,Casual,Solid,Solid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
df_fd_full.isnull().sum()

p_id                                                           0
name                                                           1
price                                                          1
colour                                                         4
brand                                                          6
ratingCount                                                 7730
avg_rating                                                  7730
Occasion                                                    1100
Pattern                                                     3806
Print or Pattern Type                                       4276
Body Shape ID                                               8026
Body or Garment Size                                        3353
Closure                                                     5783
Fabric                                                      3885
Hemline                                                     9263
Hood                     

In [84]:

# Set the threshold for the number of empty values
threshold = 0.6  # 50% empty values

# Get the number of rows in the dataframe
num_rows = df_fd_full.shape[0]

# Calculate the number of empty values in each column
null_counts = df_fd_full.isnull().sum()

# Calculate the percentage of empty values in each column
null_percentages = null_counts / num_rows

# Get the names of the columns with empty values above the threshold
columns_to_drop = null_percentages[null_percentages > threshold].index

# Print the names of the columns
print(columns_to_drop)

Index(['Hemline', 'Hood', 'Pocket', 'Bottom Fabric', 'Bottom Pattern',
       'Dupatta Border', 'Dupatta Fabric', 'Dupatta Pattern', 'Kurta Fabric',
       'Kurta Pattern', 'Ornamentation', 'Set Content', 'Stitch', 'Technique',
       'Add-Ons', 'Brand Fit Name', 'Fabric 2', 'Features', 'Fit', 'Fly Type',
       'Multipack Set', 'Type of Pleat', 'Waist Rise', 'Weave Type',
       'Distress', 'Effects', 'Fabric 3', 'Fade', 'Reversible', 'Shade',
       'Stretch', 'Type of Distress', 'Waistband', 'Center Front Open',
       'Fabric Type', 'Lining', 'Sleeve Styling', 'Transparency',
       'taxMaterial', 'Front Styling', 'Knit or Woven', 'Sport', 'Technology',
       'Blouse Closure', 'Blouse Fabric', 'Choli Stitch', 'Dupatta',
       'Lehenga Closure', 'Lehenga Fabric', 'Lehenga Lining Fabric',
       'Lehenga Stitch', 'Regions', 'Style Tip', 'Top Pattern',
       'Where-to-wear', 'Bottom Closure', 'Bottom Type', 'Top Fabric',
       'Top Type', 'Trends', 'Blouse', 'Border', 'Care for me

In [87]:
#Adjusted the columns_to_drop to remove fabric types
columns_to_drop = ['Hemline', 'Hood', 'Pocket', 'Bottom Fabric', 'Bottom Pattern',
       'Dupatta Border', 'Dupatta Fabric', 'Dupatta Pattern', 'Kurta Fabric',
       'Kurta Pattern', 'Ornamentation', 'Set Content', 'Stitch', 'Technique',
       'Add-Ons', 'Brand Fit Name', 'Features', 'Fit', 'Fly Type',
       'Multipack Set', 'Type of Pleat', 'Waist Rise', 'Weave Type',
       'Distress', 'Effects', 'Fade', 'Reversible', 'Shade',
       'Stretch', 'Type of Distress', 'Waistband', 'Center Front Open',
       'Lining', 'Sleeve Styling', 'Transparency',
       'taxMaterial', 'Front Styling', 'Knit or Woven', 'Sport', 'Technology',
       'Blouse Closure', 'Blouse Fabric', 'Choli Stitch', 'Dupatta',
       'Lehenga Closure', 'Lehenga Fabric', 'Lehenga Lining Fabric',
       'Lehenga Stitch', 'Regions', 'Style Tip', 'Top Pattern',
       'Where-to-wear', 'Bottom Closure', 'Bottom Type', 'Top Fabric',
       'Top Type', 'Trends', 'Blouse', 'Border', 'Care for me', 'Saree Fabric',
       'Collar', 'Lining Fabric', 'Pattern Coverage', 'Slit Detail',
       'Top Design Styling', 'Top Hemline', 'Top Length', 'Top Shape',
       'Weave Pattern', 'Design Styling', 'Shape', 'Sport Team',
       'Technology Present', 'Sub Trend', 'Wedding', 'Fabric Purity',
       'Fusion Wear', 'Pattern Size',
       'Contact Brand or Retailer for pre-sales product queries',
       'Colour Family', 'Collection Name', 'Better Cotton Initiative',
       'About the Brand', 'How to care for your denim',
       'What it does for your body']
print(len(columns_to_drop))

86


In [88]:
df_fd_full.drop(columns_to_drop, axis=1, inplace=True)
df_fd_full.shape

(14311, 27)

##### List of data to data to use for modelling.

df_fd_full,
df_bd,
df_fd_desc